In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import lab4_hmc as hmc

In [2]:
df_train = pd.read_csv('ee-train.csv')
df_test = pd.read_csv('ee-test.csv')

In [3]:
X_train = np.array(df_train.iloc[:, :-1])
y_train = np.array(df_train.iloc[:, -1])
X_test = np.array(df_test.iloc[:, :-1])
y_test = np.array(df_test.iloc[:, -1])

scaler = StandardScaler() # Standardise input variables

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
### EXAMPLES
def e_func(x, f):
    # Simulating some unknown log-probability
    p0 = -x[0]**2/2
    p1 = -x[1]**2/2 + np.log(2+np.cos(f*x[1]))
    lgp = p0 + p1
    return -lgp

def e_grad(x, f):
    g = np.empty(2)
    g[0] = x[0]
    g[1] = x[1] + f*np.sin(f*x[1]) / (2+np.cos(f*x[1]))
    return g

### Define Functions Here

In [5]:
# X is observation data, x is hyperparameter set sigma_w_2 and sigma_n_2

def compute_log_marginal(x, X, t): # p(t|sigma_n_2, sigma_w_2)
    sigma_w_2 = x[0]
    sigma_n_2 = x[1]
    N = X.shape[0]
    cov = sigma_n_2*np.eye(N) + sigma_w_2*X@X.T
    return stats.multivariate_normal.logpdf(t, mean=None, cov=cov, allow_singular=True)

# log uniform prior for both hyperparameters as they are scale parameters
def e_func(x, X, t):
    sigma_w_2 = x[0]
    sigma_n_2 = x[1]
    lgp = compute_log_marginal(x, X, t) + np.log(sigma_w_2) + np.log(sigma_n_2)
    return -lgp # -ve log prob

def e_grad(x, X, t):
    sigma_w_2 = x[0]
    sigma_n_2 = x[1]
    N = X.shape[0]
    cov = sigma_n_2*np.eye(N) + sigma_w_2*X@X.T
    cov_inv = np.linalg.inv(cov)
    deriv_w = X@X.T
    deriv_n = np.eye(N)
    
    grad_0 = t.T@cov_inv@deriv_w@cov_inv@t.T / 2 - np.trace(cov_inv@deriv_w) / 2+ 1/sigma_w_2
    grad_1 = t.T@cov_inv@deriv_n@cov_inv@t.T / 2 - np.trace(cov_inv@deriv_n) / 2 + 1/sigma_n_2
    
    return np.array([-grad_0, -grad_1])

In [6]:
X = X_train
t = y_train

In [8]:
x0 = np.array([0.1, 0.1]) # random point to start
hmc.gradient_check(x0, e_func, e_grad, X, t)

Calc.         Numeric       Delta         Acc.
    -5653.29      -5653.29   2.022182e-04   8
-1.02598e+07  -1.02598e+07   1.477143e-03  10
